In [117]:
import networkx as nx
import scipy as sp
import numpy as np
import random
import copy
from scipy import sparse
from random import shuffle

from numpy import linalg
from networkx.algorithms.community.label_propagation import label_propagation_communities
import collections

%matplotlib inline

In [111]:
political=nx.read_gml("polblogs.gml",label='id')

#make the graph directed with no parallels
political = nx.DiGraph(political)

#returns a set of nodes of the biggest connected component
largest = max(nx.connected_components(political.to_undirected()), key=len)

# creates the graph of the biggest connected component including direction
CC_max = political.subgraph(largest)


In [217]:
a = CC_max.copy()

convergence_factor = 0.00001

#get the b% of values
percentage=0.1

# re-index node id's to simply proccess, original id's are stored in a label.
CC_max = nx.convert_node_labels_to_integers(CC_max, first_label=0, ordering='default', label_attribute='original_graph_id')

#Returns a list with the value of the nodes
original_values = dict(CC_max.nodes(data='value'))

numbers_of_items_kept_from_percentage = round(len(a)*percentage)

precision_list = []

for i in range(9):
    
    print("Now computing nodes for:",percentage)
    print("========================")
    
    sum_of_precision = 0
    precision_results = 0

    for i in range(10):
        
        true_positive = 0
        false_positive = 0
        precision = 0
        
        # shuffle the dictionary of values to get the random nodes

        shuffled = dict(original_values.items())


        shuffled_list = list(shuffled.items())
        np.random.shuffle(shuffled_list)
        shuffled = dict(shuffled_list)


        #keep the first percentage b% of the nodes
        nodes_kept = list(shuffled.items())[:numbers_of_items_kept_from_percentage]

        nodes_kept_dictionary = dict(nodes_kept)

        nodes_to_discover = list(shuffled.items())[numbers_of_items_kept_from_percentage:]

        nodes_to_be_discovered_dictionary = dict(nodes_to_discover)

        nodes_with_known_value = list(dict(nodes_kept).keys())


        #initialize not known node values

        for key in dict(nodes_kept).keys():
            shuffled[key] = 0.5


        #the previous values holds a n x 2 matrix with the original node values
        #the first column represent the id the second one the node value
        previous_values = np.array(list(original_values.items()),dtype='f')

        #drop id's so we have a clean n x 1 matrix with the original values
        previous_values = np.delete(previous_values, 0, 1)

        #the same applies for the new values variable
        new_values = np.array(list(original_values.items()),dtype='f')

        #drop id's
        new_values = np.delete(new_values, 0, 1)

        #the previous_values and new_values will be updated inside the while loop

        # adjacency matrix of the graph
        A = nx.adjacency_matrix(CC_max)

        # a n x 1 matrix that holds how many neighbors each node has
        sum_of_neighbors = np.sum(A,axis=1)

        # numbers of nodes
        size_of_vector = np.size(sum_of_neighbors,0)


        while True:

            new_values = A * new_values

            #new_values after the multiplication is a n x 1 matrix
            #we need to change the known nodes to their original value

            for key in nodes_kept_dictionary.keys():
                new_values[key] = nodes_kept_dictionary[key]


            #no we need to divide all nodes that we are trying to predict with how many neighbors they have

            for key in nodes_to_be_discovered_dictionary.keys():

                #check for NaNs
                division = new_values[key] / sum_of_neighbors[key]

                if(division != division): 
                    new_values[key] = 0
                else:
                    new_values[key] = division


            # 1 x n Matrix
            norm_previous = np.sum(previous_values, axis=0)

            # 1 x n Matrix
            norm_new = np.sum(new_values, axis=0)


            norm_difference = abs(norm_new - norm_previous)


            if norm_difference < convergence_factor:
                break

            previous_values = new_values 

        #update the new values
        for i,obj in enumerate(new_values):
            if new_values[i] >=0.5:
                new_values[i] = 0
            else:
                new_values[i] = 1


        #Now compare original values with the values after the propagation
        
        for index in nodes_to_be_discovered_dictionary.keys():
            
            if(original_values[index] == new_values[index]):
                true_positive += 1

        false_positive = size_of_vector - numbers_of_items_kept_from_percentage - true_positive

        precision = true_positive / (true_positive + false_positive)

        sum_of_precision += precision
                
    precision_results = sum_of_precision / 10
    precision_list.append(precision_results)
    
    #update percentage
    percentage += 0.1
    
    numbers_of_items_kept_from_percentage = round(len(a)*percentage)
    


Now computing nodes for: 0.1
Now computing nodes for: 0.2
Now computing nodes for: 0.30000000000000004
Now computing nodes for: 0.4
Now computing nodes for: 0.5
Now computing nodes for: 0.6
Now computing nodes for: 0.7
Now computing nodes for: 0.7999999999999999
Now computing nodes for: 0.8999999999999999


In [218]:
print(precision_list)

[0.4453636363636363, 0.2112474437627812, 0.14654970760233918, 0.12060027285129604, 0.12307692307692308, 0.1110429447852761, 0.11389645776566756, 0.11024590163934427, 0.11147540983606556]
